# ADU Histogram Plotter
This notebook will be useful for setting the ADU cutoff, especially when we are running gasses like Xe and Kr. Fluoresence may obscure any nonlinear effects as seen in LV04, so making sure we get this right at the beginning will be helpful. Modified from Ian. From past experience, it is not good to set an upper bound as it will cause a deviation from the theory at low q, since double photon per pixel events are more likely there than at large q. Also don't set the upper bound too high, since the inelastic signal will be at a range below the input photon energy. Usually a couple hundred eV. Use `xrayscatteringtools.theory.iam.iam_compton_spectrum` to get an idea of how far this spectrum extends downward.
## First loading in modules, loading in config data from .yaml file.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import psana
import yaml
with open('config.yaml') as f:
    config = yaml.safe_load(f) # Load in configuration data

## Using the psana library to make a datasource for the specific run

In [ ]:
##########################
runNumber = 15 # A single run number goes here.
experiment = config['expNumber'] # Defaults to info in config.yaml
##########################
dsname = 'exp=%s:run=%d:smd' % (experiment , runNumber) 
ds = psana.DataSource(dsname)
events = ds.events()

## Setting up the J4M detector and pulling off images

In [ ]:
###############
n_images = 10 # Amount of images we want to pull from the run, 
###############
images = np.zeros((8,512,1024,n_images)) # Preallocation of the image array
jf = psana.Detector('jungfrau4M') # Grabbing the detector
n_filled = 0 # init
for i, evt in enumerate(ds.events()): # Loop for grabbing events
    if n_filled==n_images: # Breaking out of the loop if we reach n_images
        break
    if i%10==0: # Status updates every 10 images
        print(i)
    images[:,:,:,n_filled] = jf.calib(evt) # Grabbing the image and adding it to the array
    n_filled += 1 # Tick up after loop
print(images.shape)
hists,bins = np.histogram(images.flatten(),bins=np.linspace(0,100,200)) # Binning the flattened J4M images into the histogram variables

## Plotting it

In [ ]:
##########
cutoff = 5 # Input cutoff in keV here
##########
plt.figure(figsize=(8,5))
plt.semilogy(bins[:-1],hists)
plt.xlabel('Pixel Value (ADU)')
plt.ylabel('Counts')
plt.axvline(cutoff,linestyle='--',label=f'{cutoff} keV') # You can add vertical lines here with this line
#plt.axvline(13,linestyle='--',label='12 keV') # You can add vertical lines here with this line
plt.legend(fontsize=15)
plt.title(f'{experiment}: Run {runNumber}')
plt.show()